In [43]:
import numpy as np
np.set_printoptions(threshold=10000,suppress=True)
import pandas as pd
import warnings
import matplotlib
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [ ]:
# Chargement du fichier
def chargement_fichier(f):
    data = np.loadtxt("data/" + f, delimiter="\t", dtype=str)

    # Séparation des variables
    X = data[:, :-1].astype(float)  # 4 premières colonnes
    y = data[:, -1]                 # dernière colonne (cible)

    print("X shape :", X.shape)
    print("y shape :", y.shape)
    return X, y


In [45]:
from sklearn.model_selection import train_test_split

def decoupage_donnees(X, y):
    # Découpage stratifié
    X_A, X_T, y_A, y_T = train_test_split(
        X, y,
        test_size=1/3,
        stratify=y,
        random_state=42
    )

    print("Taille apprentissage :", X_A.shape)
    print("Taille test :", X_T.shape)
    return X_A, X_T, y_A, y_T


In [50]:
from sklearn.linear_model import Perceptron
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score

def evaluation_perceptron_multiclasse(X_A, X_T, y_A, y_T):

    encoder = LabelEncoder()

    y_A_enc = encoder.fit_transform(y_A)
    y_T_enc = encoder.transform(y_T)

    perceptron = Perceptron(
        max_iter=1000,
        eta0=0.1,
        random_state=42
    )

    perceptron.fit(X_A, y_A_enc)

    y_pred = perceptron.predict(X_T)

    print("=== PERCEPTRON MULTICLASSE ===")

    accuracy = accuracy_score(y_T_enc, y_pred)
    print("Accuracy globale :", accuracy)

    conf_matrix = confusion_matrix(y_T_enc, y_pred)
    print("Matrice de confusion :\n", conf_matrix)

    # Précision pour chaque classe
    precision = precision_score(y_T_enc, y_pred, average=None, zero_division=0)
    print("\nPrécision par classe:")
    for i, p in enumerate(precision):
        print(f"  Classe {i+1}: {p:.4f}")

    # Rappel pour chaque classe
    recall = recall_score(y_T_enc, y_pred, average=None, zero_division=0)
    print("\nRappel par classe:")
    for i, r in enumerate(recall):
        print(f"  Classe {i+1}: {r:.4f}")




In [52]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

def evaluation_mlp(X_A, X_T, y_A, y_T):
    encoder = LabelEncoder()

    y_A_enc = encoder.fit_transform(y_A)
    y_T_enc = encoder.transform(y_T)

    mlp = MLPClassifier(
        hidden_layer_sizes=(3,),
        activation='relu',
        solver='adam',
        max_iter=1000,
        random_state=10
    )

    mlp.fit(X_A, y_A_enc)
    y_pred_mlp = mlp.predict(X_T)

    print("=== MLP SANS NORMALISATION ===")

    accuracy_mlp = accuracy_score(y_T_enc, y_pred_mlp)
    print("Accuracy MLP :", accuracy_mlp)

    conf_matrix_mlp = confusion_matrix(y_T_enc, y_pred_mlp)
    print("Matrice de confusion MLP :\n", conf_matrix_mlp)

    # Précision pour chaque classe
    precision = precision_score(y_T_enc, y_pred_mlp, average=None, zero_division=0)
    print("\nPrécision par classe:")
    for i, p in enumerate(precision):
        print(f"  Classe {i+1}: {p:.4f}")

    # Rappel pour chaque classe
    recall = recall_score(y_T_enc, y_pred_mlp, average=None, zero_division=0)
    print("\nRappel par classe:")
    for i, r in enumerate(recall):
        print(f"  Classe {i+1}: {r:.4f}")

In [ ]:
from sklearn.preprocessing import StandardScaler

def evaluation_mlp_normalise(X_A, X_T, y_A, y_T):
    encoder = LabelEncoder()

    y_A_enc = encoder.fit_transform(y_A)
    y_T_enc = encoder.transform(y_T)
    scaler = StandardScaler()

    # Apprentissage de la normalisation sur A
    X_A_norm = scaler.fit_transform(X_A)

    # Application de la même normalisation sur T
    X_T_norm = scaler.transform(X_T)

    mlp_norm = MLPClassifier(
        hidden_layer_sizes=(3,),
        activation='relu',
        solver='adam',
        max_iter=1000,
        random_state=42
    )

    mlp_norm.fit(X_A_norm, y_A_enc)

    y_pred_mlp_norm = mlp_norm.predict(X_T_norm)

    print("=== MLP AVEC NORMALISATION ===")

    accuracy_mlp = accuracy_score(y_T_enc, y_pred_mlp_norm)
    print("Accuracy MLP :", accuracy_mlp)

    conf_matrix_mlp = confusion_matrix(y_T_enc, y_pred_mlp_norm)
    print("Matrice de confusion MLP :\n", conf_matrix_mlp)

    # Précision pour chaque classe
    precision = precision_score(y_T_enc, y_pred_mlp_norm, average=None, zero_division=0)
    print("\nPrécision par classe:")
    for i, p in enumerate(precision):
        print(f"  Classe {i+1}: {p:.4f}")

    # Rappel pour chaque classe
    recall = recall_score(y_T_enc, y_pred_mlp_norm, average=None, zero_division=0)
    print("\nRappel par classe:")
    for i, r in enumerate(recall):
        print(f"  Classe {i+1}: {r:.4f}")


In [60]:
# Exemple d'utilisation
def traintement(f):
    X, y = chargement_fichier(f)
    X_A, X_T, y_A, y_T = decoupage_donnees(X, y)
    evaluation_perceptron_multiclasse(X_A, X_T, y_A, y_T)
    evaluation_mlp(X_A, X_T, y_A, y_T)
    evaluation_mlp_normalise(X_A, X_T, y_A, y_T)

traintement("glass.txt")

                                                     0
0    1.52101   13.64     4.49      1.10      71.78 ...
1    1.51761   13.89     3.60      1.36      72.73 ...
2    1.51618   13.53     3.55      1.54      72.99 ...
3    1.51766   13.21     3.69      1.29      72.61 ...
4    1.51742   13.27     3.62      1.24      73.08 ...
..                                                 ...
209  1.51623   14.14     0.00      2.88      72.61 ...
210  1.51685   14.92     0.00      1.99      73.06 ...
211  1.52065   14.36     0.00      2.02      73.42 ...
212  1.51651   14.38     0.00      1.94      73.61 ...
213  1.51711   14.23     0.00      2.08      73.36 ...

[214 rows x 1 columns]
X shape: (214, 0)
y shape: (214,)
[]
['1.52101   13.64     4.49      1.10      71.78     0.06      8.75      0.00      0.00       1']


ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.